<a href="https://colab.research.google.com/github/eduardoplima/annotation-error-detection-lener-br/blob/main/aed-lener-br-pt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detecção de erros de anotação no dataset LeNER-Br

A detecção de erros de anotação (Annotation Error Detection) é uma técnica utilizada para identificar inconsistências ou rótulos incorretos em conjuntos de dados anotados manualmente. Esses erros podem comprometer a qualidade dos modelos treinados sobre esses dados, em tarefas como o reconhecimento de entidades nomeadas. Ferramentas e métodos de detecção buscam localizar essas falhas de forma automatizada, garantindo maior confiabilidade nos dados e melhor desempenho dos modelos.

Nesse notebook analisaremos o dataset LeNER-Br com o objetivo de identificar possíveis erros de anotação utilizando a técnica de confident learning, implementada pela biblioteca Cleanlab. Essa abordagem permite detectar instâncias rotuladas incorretamente com base nas previsões probabilísticas de um classificador, como veremos no código abaixo.

O dataset LeNER-Br é um corpus em português voltado para o reconhecimento de entidades nomeadas (NER) em textos jurídicos brasileiros. Desenvolvido por Luz et al. (2018), o LeNER-Br é composto exclusivamente por documentos legais, como decisões judiciais e pareceres, coletados de diversos tribunais brasileiros. Ele foi manualmente anotado para identificar entidades como pessoas, organizações, localizações e expressões temporais, além de categorias jurídicas específicas como LEGISLAÇÃO e JURISPRUDÊNCIA, que não são comuns em outros corpora do português. A descrição completa do trabalho pode ser lida no artigo disponível em https://teodecampos.github.io/LeNER-Br/luz_etal_propor2018.pdf

# Configuração do ambiente

Instalamos a biblioteca Cleanlab, que será empregada na aplicação de técnicas de confident learning para identificar possíveis erros de anotação no dataset LeNER-Br. Em seguida, importamos as bibliotecas necessárias para o restante da análise e realizamos o download dos arquivos de treinamento e teste diretamente do repositório oficial do LeNER-Br. Como os arquivos estão no formato CoNLL, que organiza os dados em colunas, é necessário convertê-los para o formato BIO (Beginning, Inside, Outside), amplamente utilizado em tarefas de reconhecimento de entidades nomeadas, para facilitar o processamento subsequente.



In [2]:
!pip install cleanlab

In [1]:
import os
import re
import requests
import zipfile
import numpy as np
import pandas as pd
import torch

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier

from transformers import AutoTokenizer, AutoModel
from cleanlab.filter import find_label_issues


In [3]:
!wget https://raw.githubusercontent.com/eduardoplima/aed-lener-br/refs/heads/main/leNER-Br/train/train.conll
!wget https://raw.githubusercontent.com/eduardoplima/aed-lener-br/refs/heads/main/leNER-Br/test/test.conll


--2025-05-17 18:06:48--  https://raw.githubusercontent.com/eduardoplima/aed-lener-br/refs/heads/main/leNER-Br/train/train.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2142199 (2.0M) [text/plain]
Saving to: ‘train.conll.4’

train.conll.4       100%[===================>]   2.04M  --.-KB/s    in 0.009s  

2025-05-17 18:06:50 (215 MB/s) - ‘train.conll.4’ saved [2142199/2142199]

--2025-05-17 18:06:50--  https://raw.githubusercontent.com/eduardoplima/aed-lener-br/refs/heads/main/leNER-Br/test/test.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting re

In [4]:
NUM_FOLDS_CV = 5
RANDOM_SEED = 43 # quase 42 😂 (e primo!)

In [5]:
def carregar_conll_lener(caminho_arquivo):
    sentencas = []
    tokens_sentenca_atual = []
    etiquetas_sentenca_atual = []

    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        for linha in f:
          linha = linha.strip()
          if linha:
              partes = linha.split()
              if len(partes) >= 2:
                  token = partes[0]
                  etiqueta_ner = partes[-1]
                  tokens_sentenca_atual.append(token)
                  etiquetas_sentenca_atual.append(etiqueta_ner)
              else:
                  pass
          else:
              if tokens_sentenca_atual:
                  sentencas.append(list(zip(tokens_sentenca_atual, etiquetas_sentenca_atual)))
                  tokens_sentenca_atual = []
                  etiquetas_sentenca_atual = []

        if tokens_sentenca_atual:
            sentencas.append(list(zip(tokens_sentenca_atual, etiquetas_sentenca_atual)))

    return sentencas

In [6]:
sentencas_treino = carregar_conll_lener("train.conll")
print(f"\nCarregadas {len(sentencas_treino)} sentenças de train.conll.")


Carregadas 7827 sentenças de train.conll.


A seguir vemos alguns exemplo de sentenças em formato BIO.

In [7]:
print("\nExemplo de sentença:")
for token, label in sentencas_treino[5][-500:]:
    print(f"{token}\t{label}")


Exemplo de sentença:
V.v	O
APELAÇÃO	O
CÍVEL	O
-	O
NULIDADE	O
PROCESSUAL	O
-	O
INTIMAÇÃO	O
DO	O
MINISTÉRIO	B-ORGANIZACAO
PÚBLICO	I-ORGANIZACAO
-	O
INCAPAZ	O
ACOMPANHADA	O
DE	O
REPRESENTANTE	O
LEGAL	O
E	O
DE	O
ADVOGADO	O
-	O
EXERCÍCIO	O
DO	O
CONTRADITÓRIO	O
E	O
DA	O
AMPLA	O
DEFESA	O
-	O
AUSÊNCIA	O
DE	O
PREJUÍZOS	O
-	O
VÍCIO	O
AFASTADO	O
-	O
IMPROCEDÊNCIA	O
DO	O
PEDIDO	O
-	O
INEXISTÊNCIA	O
DE	O
PROVA	O
QUANTO	O
AO	O
FATO	O
CONSTITUTIVO	O
DO	O
DIREITO	O
.	O


# Extração de tokens

Conforme vimos, as sentenças extraídas estão organizadas em tuplas de tokens e seus respectivos labels BIO. O objetivo é extrair as features para cada token das nossas sentenças de treinamento utilizando um modelo BERT, via Hugging Face: nesse caso, o neuralmind/bert-large-portuguese-cased. Definimos explicitamente um tamanho máximo de 512, tamanho padrão do modelo BERT escolhido, para evitar OverflowError.

O processo seguido em cada sentença é o seguinte: primeiro, extraímos os textos dos tokens. Em seguida, usamos o `tokenizer_hf` para converter esses textos em um formato que o modelo BERT entenda, especificando que a entrada já está dividida em palavras (is_split_into_words=True) e movendo os dados para o dispositivo de processamento (CPU ou GPU). Com os inputs prontos, passamos pelo model_hf para obter os "hidden states" da última camada, que são os embeddings contextuais para cada subpalavra gerada pelo tokenizador. Como o BERT trabalha com subpalavras, precisamos alinhar esses embeddings de volta aos nossos tokens originais. Para isso, utilizamos os word_ids fornecidos pelo tokenizador e, para cada token original, calculamos a média dos embeddings de suas subpalavras constituintes. Esses vetores médios são então adicionados à nossa lista final features_tokens, representando numericamente cada palavra do nosso corpus.



In [8]:
hf_model_name = "neuralmind/bert-large-portuguese-cased"

tokenizer_hf = AutoTokenizer.from_pretrained(hf_model_name)
model_hf = AutoModel.from_pretrained(hf_model_name)

In [9]:
todos_tokens = []
todos_labels_ner = []
ids_sentenca = []

for i, sentenca in enumerate(sentencas_treino):
  for token_text, ner_tag in sentenca:
    todos_tokens.append(token_text)
    todos_labels_ner.append(ner_tag)
    ids_sentenca.append(i)

print(f"\nTotal de tokens nos dados de treinamento: {len(todos_tokens)}")
print(f"Labels únicas do LeNER-Br: {sorted(list(set(todos_labels_ner)))}")


Total de tokens nos dados de treinamento: 229277
Labels únicas do LeNER-Br: ['B-JURISPRUDENCIA', 'B-LEGISLACAO', 'B-LOCAL', 'B-ORGANIZACAO', 'B-PESSOA', 'B-TEMPO', 'I-JURISPRUDENCIA', 'I-LEGISLACAO', 'I-LOCAL', 'I-ORGANIZACAO', 'I-PESSOA', 'I-TEMPO', 'O']


Temos um total de 13 labels únicas no nosso dataset. A seguir vemos uma explicação do significado de cada uma delas:

* **`B-JURISPRUDENCIA`**:
    * **B**: Indica que este token é o **início** (Beginning) de uma entidade nomeada.
    * **JURISPRUDENCIA**: Indica que a entidade nomeada é do tipo "Jurisprudência". Refere-se a decisões judiciais, acórdãos, súmulas ou qualquer conjunto de interpretações das leis feitas pelos tribunais.
    * *Exemplo*: No texto "Conforme o **Acórdão** nº 123...", "Acórdão" poderia ser `B-JURISPRUDENCIA`.

* **`B-LEGISLACAO`**:
    * **B**: Início da entidade.
    * **LEGISLACAO**: Indica que a entidade nomeada é do tipo "Legislação". Refere-se a leis, decretos, portarias, códigos, constituições, etc.
    * *Exemplo*: No texto "A **Lei** nº 8.666/93...", "Lei" poderia ser `B-LEGISLACAO`.

* **`B-LOCAL`**:
    * **B**: Início da entidade.
    * **LOCAL**: Indica que a entidade nomeada é um "Local". Pode ser uma cidade, estado, país, endereço, acidente geográfico, etc.
    * *Exemplo*: No texto "Ele viajou para **Paris**...", "Paris" seria `B-LOCAL`.

* **`B-ORGANIZACAO`**:
    * **B**: Início da entidade.
    * **ORGANIZACAO**: Indica que a entidade nomeada é uma "Organização". Inclui empresas, instituições governamentais, ONGs, times esportivos, etc.
    * *Exemplo*: No texto "O **Google** anunciou...", "Google" seria `B-ORGANIZACAO`.

* **`B-PESSOA`**:
    * **B**: Início da entidade.
    * **PESSOA**: Indica que a entidade nomeada é uma "Pessoa". Refere-se a nomes de indivíduos.
    * *Exemplo*: No texto "**Maria** Silva é advogada...", "Maria" seria `B-PESSOA`.

* **`B-TEMPO`**:
    * **B**: Início da entidade.
    * **TEMPO**: Indica que a entidade nomeada é uma referência temporal. Pode ser uma data, hora, período específico (ex: "século XXI", "próxima semana").
    * *Exemplo*: No texto "A reunião será em **15 de maio**...", "15" poderia ser `B-TEMPO`.

* **`I-JURISPRUDENCIA`**:
    * **I**: Indica que este token está **dentro** (Inside) de uma entidade do tipo "Jurisprudência" que já começou. É uma continuação da entidade.
    * *Exemplo*: No texto "...o **Superior Tribunal** de Justiça...", se "Superior" foi `B-JURISPRUDENCIA` (ou `B-ORGANIZACAO` dependendo do esquema), "Tribunal" poderia ser `I-JURISPRUDENCIA` (ou `I-ORGANIZACAO`). No caso de um nome de jurisprudência longo, como "Súmula **Vinculante nº** 56", "Vinculante", "nº" e "56" seriam `I-JURISPRUDENCIA` se "Súmula" fosse `B-JURISPRUDENCIA`.

* **`I-LEGISLACAO`**:
    * **I**: Dentro de uma entidade do tipo "Legislação".
    * *Exemplo*: No texto "A **Lei de Licitações**...", se "Lei" foi `B-LEGISLACAO`, "de" e "Licitações" seriam `I-LEGISLACAO`.

* **`I-LOCAL`**:
    * **I**: Dentro de uma entidade do tipo "Local".
    * *Exemplo*: No texto "Ele mora em **Nova York**...", se "Nova" foi `B-LOCAL`, "York" seria `I-LOCAL`.

* **`I-ORGANIZACAO`**:
    * **I**: Dentro de uma entidade do tipo "Organização".
    * *Exemplo*: No texto "O **Banco Central** do Brasil...", se "Banco" foi `B-ORGANIZACAO`, "Central" seria `I-ORGANIZACAO`.

* **`I-PESSOA`**:
    * **I**: Dentro de uma entidade do tipo "Pessoa".
    * *Exemplo*: No texto "**Maria Joaquina** da Silva...", se "Maria" foi `B-PESSOA`, "Joaquina" seria `I-PESSOA`.

* **`I-TEMPO`**:
    * **I**: Dentro de uma entidade do tipo "Tempo".
    * *Exemplo*: No texto "A reunião será em **15 de maio de 2025**...", se "15" foi `B-TEMPO`, "de", "maio", "de" e "2025" seriam `I-TEMPO`.

* **`O`**:
    * **O**: Indica que o token está **fora** (Outside) de qualquer entidade nomeada. É um token comum que não faz parte de uma categoria de interesse específica.
    * *Exemplo*: No texto "O gato **sentou** no tapete.", "sentou" seria `O`.


In [10]:
import numpy as np
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando device: {device}")

EFFECTIVE_MAX_LENGTH = 512

model_hf.to(device)
model_hf.eval()

features_tokens = []

for i_sent, dados_sentenca in enumerate(sentencas_treino):
    textos_sentenca = [dado_token[0] for dado_token in dados_sentenca]

    if not textos_sentenca:
        # Sentença vazia, pulamos
        continue

    inputs = tokenizer_hf(
        textos_sentenca,
        is_split_into_words=True,
        return_tensors="pt",
        padding="longest",
        truncation=True,
        max_length=EFFECTIVE_MAX_LENGTH
    ).to(device)

    word_ids = inputs.word_ids()
    with torch.no_grad():
        outputs = model_hf(**inputs)
        last_hidden_state = outputs.last_hidden_state

    token_subword_embeddings = [[] for _ in range(len(textos_sentenca))]

    for subword_idx, original_word_idx in enumerate(word_ids):
        if original_word_idx is not None: # Exclui tokens como [CLS], [SEP] ou tokens de padding
            embedding = last_hidden_state[0, subword_idx, :]
            token_subword_embeddings[original_word_idx].append(embedding)

    sentence_token_features = []
    for original_token_idx in range(len(textos_sentenca)):
        if token_subword_embeddings[original_token_idx]:
            stacked_embeddings = torch.stack(token_subword_embeddings[original_token_idx])
            mean_embedding = torch.mean(stacked_embeddings, dim=0)
            sentence_token_features.append(mean_embedding.cpu().numpy())
        else:
            sentence_token_features.append(np.zeros(model_hf.config.hidden_size))

    features_tokens.extend(sentence_token_features)


Usando device: cuda


In [11]:
features_tokens = np.array(features_tokens)
print(f"Formato da matriz de características dos tokens: {features_tokens.shape}")

Formato da matriz de características dos tokens: (229277, 1024)


# Treino do modelo

Precisamos treinar um modelo para uso na biblioteca cleanlab. Inicialmente, os labels NER são transformados utilizando a LabelEncoder, que transforma os rótulos em números para uso no modelo de Regressão Logística escolhido.

No efetivo treino do nosso modelo dividimos nosso conjunto de dados (`features_tokens` e `labels_ner_codificados`) em duas partes: uma para treinamento (`X_treino`,`y_treino`) e outra para teste (`X_teste`, `y_teste`). Utilizamos 25% dos dados para o conjunto de teste e garantimos que a proporção das diferentes classes de rótulos NER seja mantida em ambas as divisões, graças ao parâmetro stratify. Em seguida, preparamos um array chamado `probabilidades_preditas_teste`, que servirá para armazenar as probabilidades de cada classe que o nosso modelo atribuirá aos exemplos do conjunto de teste.

Em seguida, definimos e treinamos o nosso modelo de classificação. Optamos por um SGDClassifier (Stochastic Gradient Descent Classifier). Ele funciona ajustando os parâmetros de um modelo linear (neste caso, configurado para se comportar como uma Regressão Logística usando loss='log_loss') de forma iterativa, processando uma amostra por vez, fazendo com que seja rápido e escalável. Após treinar o modelo, nós o utilizamos para prever as probabilidades das classes para o conjunto `X_teste`, armazenando-as em `probabilidades_preditas_teste`. Por fim, também calculamos e exibimos a acurácia do modelo nesse conjunto de teste, comparando as predições com os rótulos verdadeiros y_teste.

Uma estratégia de KFold com 5 folds foi utilizada para percorrer o dataset inteiro de modo separado e independente.

In [12]:
label_encoder = LabelEncoder()
labels_ner_codificados = label_encoder.fit_transform(todos_labels_ner)
num_classes = len(label_encoder.classes_)

In [22]:
skf = StratifiedKFold(n_splits=NUM_FOLDS_CV, shuffle=True, random_state=RANDOM_SEED)
probabilidades_preditas = np.zeros((len(features_tokens), num_classes))
print(f"\nIniciando validação cruzada de {NUM_FOLDS_CV} folds")

for indice_fold, (indices_treino, indices_validacao) in enumerate(skf.split(features_tokens, labels_ner_codificados)):
    print(f"  Processando Fold {indice_fold + 1}/{NUM_FOLDS_CV}...")
    X_treino, X_validacao = features_tokens[indices_treino], features_tokens[indices_validacao]
    y_treino, y_validacao = labels_ner_codificados[indices_treino], labels_ner_codificados[indices_validacao]

    modelo = SGDClassifier(
            loss='log_loss',
            penalty='l2',
            alpha=0.0001,
            max_iter=1000,
            tol=1e-3,
            random_state=RANDOM_SEED,
            class_weight='balanced',
            learning_rate='optimal',
            early_stopping=True,
            n_iter_no_change=10,
            validation_fraction=0.1
        )


    modelo.fit(X_treino, y_treino)
    print("Modelo treinado.")
    probabilidades_preditas_fold = modelo.predict_proba(X_validacao)
    probabilidades_preditas[indices_validacao] = probabilidades_preditas_fold

    predicoes_fold = modelo.predict(X_validacao)
    acuracia_fold = accuracy_score(y_validacao, predicoes_fold)
    print(f"    Acurácia do Fold {indice_fold + 1}: {acuracia_fold:.4f}")

print("\nColeta de probabilidades preditas fora da amostra finalizada.")
print(f"Formato da matriz de probabilidades_preditas: {probabilidades_preditas.shape}")



Iniciando validação cruzada de 5 folds
  Processando Fold 1/5...
Modelo treinado.
    Acurácia do Fold 1: 0.9680
  Processando Fold 2/5...
Modelo treinado.
    Acurácia do Fold 2: 0.9734
  Processando Fold 3/5...
Modelo treinado.
    Acurácia do Fold 3: 0.9696
  Processando Fold 4/5...
Modelo treinado.
    Acurácia do Fold 4: 0.9720
  Processando Fold 5/5...
Modelo treinado.
    Acurácia do Fold 5: 0.9719

Coleta de probabilidades preditas fora da amostra finalizada.
Formato da matriz de probabilidades_preditas: (229277, 13)


# Problemas encontrados com o dataset

Agora adentramos na aplicação de fato das técnicas de confident learning. Inicialmente, utilizamos a função `find_label_issues` da biblioteca cleanlab para identificar tokens potencialmente mal rotulados em nosso dataset de NER. Passamos como entrada os label codificados (`labels_ner_codificados`) e as probabilidades previstas pelo modelo (`probabilidades_preditas`).

In [23]:
len(labels_ner_codificados),len(probabilidades_preditas)

(229277, 229277)

In [24]:
print("\nIdentificando problemas de rotulagem com cleanlab...")

indices_problemas_rotulos = find_label_issues(
        labels=labels_ner_codificados,
        pred_probs=probabilidades_preditas,
        return_indices_ranked_by='self_confidence'
    )

num_problemas_encontrados = len(indices_problemas_rotulos)
print(f"Cleanlab identificou {num_problemas_encontrados} potenciais problemas de rotulagem.")
percentual_problemas = (num_problemas_encontrados / len(todos_tokens)) * 100
print(f"Isso representa {percentual_problemas:.2f}% do total de tokens.")


Identificando problemas de rotulagem com cleanlab...
Cleanlab identificou 2326 potenciais problemas de rotulagem.
Isso representa 1.01% do total de tokens.



Em seguida, percorremos os índices dos tokens que apresentam possíveis erros de anotação no dataset de NER, comparando os rótulos originais com os rótulos sugeridos pelo modelo. Para cada token identificado como problemático, recuperamos o token, o label e o transformamos para sua forma textual usando o label_encoder (método `inverse_transform`).

Entção, identificamos o rótulo predito pelo modelo com maior probabilidade e também o decodificamos. Calculamos a confiança do modelo no rótulo original e recuperamos o identificador da sentença a que o token pertence. Por fim, reunimos todas essas informações em uma lista de `dicts` (`problemas_para_revisao`).

O `dict`  armazenado possui os seguintes campos que serão úteis para nossa análise posterior:

* `indice_token_global`: posição do token em nossa lista com todos os tokens do nosso dataset
* `id_sentenca`: identificador da frase problemática
* `rotulo_original`: o label que veio associado ao token no dataset
* `rotulo_sugerido_pelo_modelo`: o label que nosso modelo sugere para o token
* `confianca_modelo_no_rotulo_original`: a probabilidade que o modelo atribui ao rótulo original. Valores baixos significam que nosso modelo não tem muita confiança que o label original está correto.
* `contexto_sentenca_completa`: sentença completa onde o token problemático foi encontrado. Será utilizado para visualizarmos os problemas que serão tratados em um passo posterior.


In [25]:
problemas_para_revisao = []
for indice_token_global in indices_problemas_rotulos:
    token_original = todos_tokens[indice_token_global]
    rotulo_original_codificado = labels_ner_codificados[indice_token_global]
    rotulo_original_str = label_encoder.inverse_transform([rotulo_original_codificado])[0]
    rotulo_predito_codificado = np.argmax(probabilidades_preditas[indice_token_global])
    rotulo_predito_str = label_encoder.inverse_transform([rotulo_predito_codificado])[0]

    confianca_no_original = probabilidades_preditas[indice_token_global, rotulo_original_codificado]

    id_sent = ids_sentenca[indice_token_global]

    problemas_para_revisao.append({
        "indice_token_global": indice_token_global,
        "id_sentenca": id_sent,
        "token": token_original,
        "rotulo_original": rotulo_original_str,
        "rotulo_sugerido_pelo_modelo": rotulo_predito_str,
        "confianca_modelo_no_rotulo_original": confianca_no_original,
        "contexto_sentenca_completa": sentencas_treino[id_sent]
    })

Ordenamos os problemas pelas menores confianças do modelo nos rótulos fornecidos originalmente

In [30]:
problemas_para_revisao_ordenados = sorted(problemas_para_revisao, key=lambda x: x['confianca_modelo_no_rotulo_original'])

E visualizamos os problemas encontrados. No laço a seguir temos os 20 problemas com menor confianca do modelo no rótulo original, ou seja, maior desconfiança.

In [31]:
for i, problema in enumerate(problemas_para_revisao_ordenados[:min(20, num_problemas_encontrados)]):
    print(f"\nProblema #{i+1} (Índice Global do Token: {problema['indice_token_global']})")
    print(f"  ID da Sentença: {problema['id_sentenca']}")
    print(f"  Token: '{problema['token']}'")
    print(f"  Rótulo Original: {problema['rotulo_original']}")
    print(f"  Rótulo Sugerido pelo Modelo: {problema['rotulo_sugerido_pelo_modelo']}")
    print(f"  Confiança do Modelo no Rótulo Original: {problema['confianca_modelo_no_rotulo_original']:.4f}")

    tokens_tags_sentenca = problema['contexto_sentenca_completa'] # Lista de tuplas (token_texto, rotulo_original)

    idx_primeiro_token_da_sentenca_no_dataset_global = -1
    for idx_global, sent_id_global in enumerate(ids_sentenca):
        if sent_id_global == problema['id_sentenca']:
            idx_primeiro_token_da_sentenca_no_dataset_global = idx_global
            break

    posicao_token_na_sentenca = problema['indice_token_global'] - idx_primeiro_token_da_sentenca_no_dataset_global

    if not (0 <= posicao_token_na_sentenca < len(tokens_tags_sentenca)) or \
       tokens_tags_sentenca[posicao_token_na_sentenca][0] != problema['token']:
        for idx_sent, (tk_sent, _) in enumerate(tokens_tags_sentenca):
            if tk_sent == problema['token']:
                posicao_token_na_sentenca = idx_sent
                break
    janela_contexto = 10

    inicio_ctx_ant = max(0, posicao_token_na_sentenca - janela_contexto)
    contexto_anterior_dados = tokens_tags_sentenca[inicio_ctx_ant : posicao_token_na_sentenca]
    contexto_anterior_formatado = [f"{tk}({tag})" for tk, tag in contexto_anterior_dados]

    token_problematico_texto = problema['token']
    rotulo_original_problematico = problema['rotulo_original']
    rotulo_sugerido_problematico = problema['rotulo_sugerido_pelo_modelo']
    token_destacado_str = f"**{token_problematico_texto}**(Original:{rotulo_original_problematico}|Sugerido:{rotulo_sugerido_problematico})**"

    inicio_ctx_post = posicao_token_na_sentenca + 1
    fim_ctx_post = min(len(tokens_tags_sentenca), inicio_ctx_post + janela_contexto)
    contexto_posterior_dados = tokens_tags_sentenca[inicio_ctx_post : fim_ctx_post]
    contexto_posterior_formatado = [f"{tk}({tag})" for tk, tag in contexto_posterior_dados]

    partes_finais_contexto = []
    if contexto_anterior_formatado:
        partes_finais_contexto.append(" ".join(contexto_anterior_formatado))

    partes_finais_contexto.append(token_destacado_str)

    if contexto_posterior_formatado:
        partes_finais_contexto.append(" ".join(contexto_posterior_formatado))

    print(f"  Contexto (±{janela_contexto} palavras): {' '.join(partes_finais_contexto)}")

print("\nFim da exibição dos problemas.")



Problema #1 (Índice Global do Token: 138519)
  ID da Sentença: 4905
  Token: 'artigo'
  Rótulo Original: B-LOCAL
  Rótulo Sugerido pelo Modelo: B-LEGISLACAO
  Confiança do Modelo no Rótulo Original: 0.0000
  Contexto (±10 palavras): Logo(O) ,(O) tem-se(O) que(O) o(O) **artigo**(Original:B-LOCAL|Sugerido:B-LEGISLACAO)** 276(I-LOCAL) do(I-LOCAL) Decreto(I-LOCAL) nº(I-LOCAL) 3.048/99(I-LOCAL) especificamente(O) fixa(O) o(O) dia(O) dois(O)

Problema #2 (Índice Global do Token: 122878)
  ID da Sentença: 4323
  Token: 'Autos'
  Rótulo Original: B-LOCAL
  Rótulo Sugerido pelo Modelo: B-JURISPRUDENCIA
  Confiança do Modelo no Rótulo Original: 0.0000
  Contexto (±10 palavras): 68(O) 3302-0444/0445(O) ,(O) Rio(B-LOCAL) Branco-AC(I-LOCAL) -(O) Mod(O) .(O) 500258(O) -(O) **Autos**(Original:B-LOCAL|Sugerido:B-JURISPRUDENCIA)** n.º(I-LOCAL) 1002199-81.2017.8.01.0000/50000(I-LOCAL) ARAÚJO(O) ,(O) QUARTA(B-ORGANIZACAO) TURMA(I-ORGANIZACAO) ,(O) julgado(O) em(O) 09/05/2017(B-TEMPO)

Problema #3 (Índic

Nesse momento analisamos a saída do nosso modelo de confident learning. Vemos que nos primeiros problemas identificados o modelo acertadamente apontou erros na anotação humano. Os problemas #1 e #2 tratam-se claramente de exemplos de legislação cadastrados erroneamente: \*\*artigo\*\*(Original:B-LOCAL|Sugerido:B-LEGISLACAO)\*\* 276(I-LOCAL) e  \*\*Autos\*\*(Original:B-LOCAL|Sugerido:B-JURISPRUDENCIA)\*\* n.º(I-LOCAL) 1002199-81.2017.8.01.0000/50000(I-LOCAL).

Todavia, há exemplos em que nosso modelo se confundiu ao apontar problemas em labels originais. No problema #3, a vírgula no endereço QUADRA(B-LOCAL) 1(I-LOCAL) \*\* , \*\*(Original:I-LOCAL|Sugerido:O)** DO(I-LOCAL) SETOR(I-LOCAL) DE(I-LOCAL) INDÚSTRIAS(I-LOCAL) GRÁFICAS(I-LOCAL) deve ser, de fato, considerada parte do label LOCAL.

Não obstante o exemplo de engano cometido pelo modelo, percebe-se a eficiência do modelo em identificar os labels problemáticos, atestando a eficácia da técnica aplicada.



# Conclusão

Neste notebook, aplicamos técnicas de Confident Learning utilizando a biblioteca cleanlab para detectar erros de anotação no dataset LeNER-Br, amplamente utilizado em tarefas de Reconhecimento de Entidades Nomeadas (NER) em língua portuguesa.

Identificamos automaticamente diversos rótulos inconsistentes entre as anotações humanas e as previsões do modelo treinado, com base em critérios de baixa confiança. Foi possível observar que muitos dos erros apontados pelo modelo indicavam de fato falhas de rotulagem no conjunto original, como a anotação equivocada de expressões legais e nomes de jurisprudência como localidades.

Ainda que alguns falsos positivos tenham sido identificados — como o caso da vírgula no endereço classificada incorretamente pelo modelo — os resultados demonstram a relevância da técnica para auditoria e refinamento de datasets anotados manualmente.

Concluímos que o uso de Confident Learning representa uma abordagem eficaz para a melhoria da qualidade de conjuntos de dados anotados, sobretudo em tarefas sensíveis como o NER jurídico, onde erros de anotação podem impactar significativamente o desempenho dos modelos.

Como etapa futura, recomenda-se a aplicação de técnicas de retagging automatizado ou semiautomático para corrigir os rótulos identificados como problemáticos, utilizando as previsões de maior confiança do modelo como sugestão inicial para revisão humana.